In [3]:
from os import path, listdir
import sys
sys.path.append("..")

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from src.tracking import *

In [2]:
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory('../DB/test', batch_size = 3568, class_mode = 'categorical', target_size = (227, 227))

# On récupère les images d'un batch de la taille du dataset
i = 0
for image, lab in test_generator:
    if i == 0: break # sinon le generator continue sa boucle (??)

Found 3568 images belonging to 5 classes.


# Tracking vidéo
En partant d'une première détection de l'objet d'intérêt qui prend la forme d'une boite englobante que l'on suppose correcte, on va suivre l'objet sur les frames suivantes.

Pour chaque frame, on prend la boîte englobante de la frame précédente, on va créer un set de nouvelles boîtes potentielles. On va ensuite récupérer le "patch" de l'image associé à chaque nouvelle boîte englobante et effectuer une prediction à l'aide du modèle précédemment créé. On conservera comme nouvelle boîte celle qui fournira la prédiction la plus précise (pourcentage le plus élevé).

In [9]:
# on charge le modèle
model = tf.keras.models.load_model('../models/best_models', custom_objects=None, compile=True, options=None)

In [ ]:
# on vérifie les performances sur le jeu de test
model.evaluate(image, lab)

In [11]:
ious = []
i = 0
# Pour toutes les vidéos qui sont dans le jeu de test
for file in listdir("../GT_test"):
    if file.endswith(".txt"):
        i += 1
        print(f'Video {i} on {len(listdir("../GT_test"))}')
        
        m = re.search('(.+)_._bboxes\.txt', file)
        name = m.group(1)
        
        path_video = path.join('..', 'VIDEOS', name + '.mp4')
        path_boxes = path.join('..', 'GT_test', file)
        
        label = get_label(path_video)
        predictions, iou = tracking(model, path_video, path_boxes, label, update=None, save_fig=True)
        mean_iou = np.mean(list(iou.values()))
        ious.append(mean_iou)
        print(f'Mean IoU: {np.mean(ious_move_to_data)}')
            
print(f'Total Mean IoU: {np.mean(ious)}')



  0%|          | 0/186 [00:00<?, ?it/s]

  9%|▉         | 17/186 [00:00<00:01, 167.62it/s]

Video 1 on 27
Loading 186 frames




 22%|██▏       | 41/186 [00:00<00:00, 183.55it/s]

 33%|███▎      | 61/186 [00:03<00:07, 16.90it/s] 

 33%|███▎      | 61/186 [00:17<00:07, 16.90it/s]

 35%|███▍      | 65/186 [00:17<02:06,  1.05s/it]

 35%|███▌      | 66/186 [00:20<03:29,  1.75s/it]

 36%|███▌      | 67/186 [00:24<04:29,  2.26s/it]

 37%|███▋      | 68/186 [00:27<05:07,  2.60s/it]

 37%|███▋      | 69/186 [00:31<05:33,  2.85s/it]

 38%|███▊      | 70/186 [00:34<05:47,  2.99s/it]

 38%|███▊      | 71/186 [00:37<05:58,  3.11s/it]

 39%|███▊      | 72/186 [00:41<06:03,  3.19s/it]

 39%|███▉      | 73/186 [00:44<06:05,  3.23s/it]

 40%|███▉      | 74/186 [00:47<06:03,  3.25s/it]

 40%|████      | 75/186 [00:50<06:01,  3.26s/it]

 41%|████      | 76/186 [00:54<05:59,  3.26s/it]

 41%|████▏     | 77/186 [00:57<05:56,  3.27s/it]

 42%|████▏     | 78/186 [01:00<05:54,  3.29s/it]

 42%|████▏     | 79/186 [01:04<05:52,  3.29s/it]

 43%|████▎     | 80/186 [01:07<05:49,  3.29s/it]

 44%|████▎     | 81/186 [01:10<05:46,  3.30s/i

NameError: name 'ious_move_to_data' is not defined

## Move-to-data
Nous implémentons ici la méthode d'apprentissage incrémental "move-to-data" décrite dans _Move-to-Data: A new Continual Learning approach with Deep CNNs, Application for image-class recognition_

Pour chaque nouvelle image, on met à jour les poids de la dernière couche du réseau en appliquant la formule suivante 
$$w_j′=w_j+(||w_j||∗\frac{v_i}{||v_i||} − w_j)*\epsilon$$

In [4]:
ious_move_to_data = []
i = 0
# Pour toutes les vidéos qui sont dans le jeu de test
for file in listdir("../GT_test"):
    if file.endswith(".txt"):
        i += 1
        print(f'Video {i} on {len(listdir("../GT_test"))}')
        
        m = re.search('(.+)_._bboxes\.txt', file)
        name = m.group(1)
        
        path_video = path.join('..', 'VIDEOS', name + '.mp4')
        path_boxes = path.join('..', 'GT_test', file)
        
        label = get_label(path_video)
        
        # on lance le tracking en activant le move-to-data, on met à jour le réseau toute les 10 images
        predictions, iou = tracking(model, path_video, path_boxes, label, update='move-to-data', epsilon=0.0001, n_update=10, save_fig=True)
        mean_iou = np.mean(list(iou.values()))
        ious_move_to_data.append(mean_iou)
        print(f'Mean IoU: {np.mean(ious_move_to_data)}')

print(f'Total Mean IoU: {np.mean(ious_move_to_data)}')

  9%|▉         | 17/186 [00:00<00:01, 165.55it/s]

Video 1 on 27
Loading 186 frames


  8%|▊         | 15/189 [00:00<00:01, 145.88it/s]

Video 2 on 27
Loading 189 frames


 10%|▉         | 20/205 [00:00<00:00, 192.98it/s]

...

Video 17 on 27
Loading 205 frames


  0%|          | 0/195 [00:00<?, ?it/s]

Video 18 on 27
Loading 195 frames


 17%|█▋        | 33/195 [00:57<01:46,  1.53it/s]

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:
# on évalue les modifications apportées au modèle à l'aide des images de test
model.evaluate(image, lab)  

## Fine-tuning
Pour comparer la méthode move-to-data on implémente une méthode déjà éprouvée, le fine-tuning consiste à réentrainer la dernière couche du réseau avec de nouveaux batch d'image

In [ ]:
# on recharge le modèle pour ne pas écraser celui modifié par le move-to-data
model_fine = tf.keras.models.load_model('../models/best_models', custom_objects=None, compile=True, options=None)

ious_fine_tuning = []
i = 0
# Pour toutes les vidéos qui sont dans le jeu de test
for file in listdir("../GT_test"):
    if file.endswith(".txt"):
        i += 1
        print(f'Video {i} on {len(listdir("../GT_test"))}')
        
        m = re.search('(.+)_._bboxes\.txt', file)
        name = m.group(1)
        
        path_video = path.join('..', 'VIDEOS', name + '.mp4')
        path_boxes = path.join('..', 'GT_test', file)
        
        label = get_label(path_video)
        
        # on lance le tracking en activant le fine-tuning, on met à jour le réseau toutes les 10 images
        predictions, iou = tracking(model_fine, path_video, path_boxes, label, update='fine-tuning', epsilon=0.0001, n_update=10, save_fig=False)
        mean_iou = np.mean(list(iou.values()))
        ious_fine_tuning.append(mean_iou)
        
print(f'Mean IoU: {np.mean(ious_fine_tuning)}')

In [ ]:
# on évalue les modifications apportées au modèle à l'aide des images de test
model_fine.evaluate(image, lab)  

## Comparaisons
Dans cette partie, on compare les résultats que nous avons obtenu en tracking simple (sans mise à jour) avec les résultats que nous aurions obtenu en utilisant les modèles fournis. Cette comparaison ne sera faite que sur une seule vidéo du fait d'un temps de calcul relativement long.

In [3]:
model = tf.keras.models.load_model('../models/mobilenet_based__16.h5', custom_objects=None, compile=True, options=None)
label = get_label('../VIDEOS/RicePlace6Subject4.mp4')
_, iou = tracking(model, '../VIDEOS/RicePlace6Subject4.mp4', '../GT_test/RicePlace6Subject4_2_bboxes.txt', label, save_fig=True, name='mobilenet')
mean_iou = np.mean(list(iou.values()))

print(f'Mean IoU: {mean_iou}')


 10%|▉         | 17/177 [00:00<00:00, 168.65it/s]

Loading 177 frames



100%|██████████| 177/177 [08:40<00:00,  2.94s/it]

Mean IoU: 0.06775632032573965


In [4]:
model = tf.keras.models.load_model('../models/mobilenetv2_based__6.h5', custom_objects=None, compile=True, options=None)
label = get_label('../VIDEOS/RicePlace6Subject4.mp4')
_, iou = tracking(model, '../VIDEOS/RicePlace6Subject4.mp4', '../GT_test/RicePlace6Subject4_2_bboxes.txt', label, save_fig=True, name='mobilenetV2')
mean_iou = np.mean(list(iou.values()))

print(f'Mean IoU: {mean_iou}')


 11%|█         | 19/177 [00:00<00:00, 189.42it/s]

Loading 177 frames



100%|██████████| 177/177 [08:56<00:00,  3.03s/it]

Mean IoU: 0.04415279712140601


### OpenCV Tracker
La bibliothèque OpenCV propose nativement des trackers que nous avons également comparé au notre. Nous en avons essayé trois, qui présentent des performances différentes.

In [6]:
tracker_open_cvmodel('../VIDEOS/RicePlace6Subject4.mp4', '../GT_test/RicePlace6Subject4_2_bboxes.txt', 'kcf')


  8%|▊         | 14/177 [00:00<00:01, 137.35it/s]

Loading 177 frames



100%|██████████| 177/177 [00:01<00:00, 121.49it/s]


0.25488313131166607

In [7]:
tracker_open_cvmodel('../VIDEOS/RicePlace6Subject4.mp4', '../GT_test/RicePlace6Subject4_2_bboxes.txt', 'csrt')


 10%|█         | 18/177 [00:00<00:00, 179.42it/s]

Loading 177 frames



100%|██████████| 177/177 [00:07<00:00, 22.54it/s]


0.43499035966551447

In [8]:
tracker_open_cvmodel('../VIDEOS/RicePlace6Subject4.mp4', '../GT_test/RicePlace6Subject4_2_bboxes.txt', 'mil')


 11%|█▏        | 20/177 [00:00<00:00, 195.67it/s]

Loading 177 frames



100%|██████████| 177/177 [00:07<00:00, 25.18it/s]


0.29130459375165046

In [12]:
i = 0
for file in listdir("../GT_test"):
    if file.endswith(".txt"):
        print(f'Video {i} on {len(listdir("../"))} - {file}')
        i += 1

Video 0 on 17 - CanOfCocaColaPlace5Subject2_2_bboxes.txt
Video 1 on 17 - RicePlace7Subject3_2_bboxes.txt
Video 2 on 17 - BowlPlace5Subject3_2_bboxes.txt
Video 3 on 17 - MilkBottlePlace4Subject4_2_bboxes.txt
Video 4 on 17 - MilkBottlePlace1Subject4_2_bboxes.txt
Video 5 on 17 - SugarPlace2Subject2_2_bboxes.txt
Video 6 on 17 - RicePlace2Subject2_2_bboxes.txt
Video 7 on 17 - RicePlace6Subject4_2_bboxes.txt
Video 8 on 17 - SugarPlace6Subject4_2_bboxes.txt
Video 9 on 17 - BowlPlace1Subject4_2_bboxes.txt
Video 10 on 17 - RicePlace3Subject3_2_bboxes.txt
Video 11 on 17 - BowlPlace4Subject4_2_bboxes.txt
Video 12 on 17 - SugarPlace3Subject5_2_bboxes.txt
Video 13 on 17 - RicePlace5Subject2_2_bboxes.txt
Video 14 on 17 - CanOfCocaColaPlace7Subject3_2_bboxes.txt
Video 15 on 17 - CanOfCocaColaPlace2Subject4_2_bboxes.txt
Video 16 on 17 - BowlPlace5Subject3_2_bboxes.txt_pred.txt
Video 17 on 17 - SugarPlace5Subject3_2_bboxes.txt
Video 18 on 17 - MilkBottlePlace2Subject5_2_bboxes.txt
Video 19 on 17 - CanO

In [ ]:
file = 'SugarPlace5Subject3_2_bboxes.txt'
model = tf.keras.models.load_model('../models/best_models', custom_objects=None, compile=True, options=None)

m = re.search('(.+)_._bboxes\.txt', file)
name = m.group(1)

path_video = path.join('..', 'VIDEOS', name + '.mp4')
path_boxes = path.join('..', 'GT_test', file)

label = get_label(path_video)
predictions, iou = tracking(model, path_video, path_boxes, label, update=None, save_fig=True)

  5%|▌         | 10/195 [00:00<00:01, 98.27it/s]

Loading 195 frames


 32%|███▏      | 62/195 [01:35<10:59,  4.96s/it] 